In [1]:
import gc
import time
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import imblearn.over_sampling as ovs

from sklearn.metrics import log_loss
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedKFold
from sklearn.decomposition import PCA, TruncatedSVD, LatentDirichletAllocation
from sklearn.preprocessing import StandardScaler


from sklearn.ensemble import *
from sklearn.cluster import KMeans
from sklearn.multiclass import OneVsRestClassifier
from sklearn.neural_network import MLPClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')

path = "./"
seed = 21 
le = LabelEncoder()

/home/koleshjr/.local/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
Train = pd.read_csv(path+'Train.csv')
Test = pd.read_csv(path+'Test.csv')
Sub = pd.read_csv(path+'SampleSubmission.csv')

display(Train.head(5), Test.head(5), Sub.head(5))
Train.shape, Test.shape, Sub.shape

,DATE,TRANSACTION DETAILS,Account_NO,CHQ.NO.,WITHDRAWAL AMT,DEPOSIT AMT,BALANCE AMT,Category
0,2014-02-28,NEFT/FDRL401249529/INDIAFORENSIC,84903292,NaN,0.0,15000000.0,15000000.0,Money-Transfer
1,2014-02-28,NEFT/FDRL401249532/INDIAFORENSIC,84903292,NaN,0.0,15000000.0,60000000.0,Money-Transfer
2,2014-02-28,NEFT/FDRL401249534/INDIAFORENSIC,84903292,NaN,0.0,15000000.0,90000000.0,Money-Transfer
3,2014-02-28,NEFT/FDRL401249590/INDIAFORENSIC,84903292,NaN,0.0,10000000.0,100000000.0,Money-Transfer
4,2014-02-28,NEFT/FDRL401249531/INDIAFORENSIC,84903292,NaN,0.0,15000000.0,45000000.0,Money-Transfer


,DATE,TRANSACTION DETAILS,Account_NO,CHQ.NO.,WITHDRAWAL AMT,DEPOSIT AMT,BALANCE AMT,ID
0,2014-02-28,NEFT/FDRL401249530/INDIAFORENSIC,84903292,NaN,0.00,15000000.0,30000000.00,2014-02-28 00:00:00X3000000000
1,2014-02-28,NEFT/FDRL401249533/INDIAFORENSIC,84903292,NaN,0.00,15000000.0,75000000.00,2014-02-28 00:00:00X7500000000
2,2014-03-08,RTGSCHARGESANDSTAX/RAT,84903292,NaN,28.63,0.0,45000028.63,2014-03-08 00:00:00X4500002863
3,2014-03-08,RTGS/YESBH16068986176/INDFOR,84903292,NaN,15000000.00,0.0,60000028.63,2014-03-08 00:00:00X6000002863
4,2014-03-08,RTGSCHARGESANDSTAX/RAT,84903292,NaN,28.63,0.0,60000057.26,2014-03-08 00:00:00X6000005726


,ID,Bank Charges,Bill-Payments,Cash-Pickup,Cell Phone and Airtime,Cheque-Payment,Deposit,Donations,General Purchases,Insurance,Interest,Internet and IT Services,Loan Repayment,Merchant-Payment,Money-Transfer,Professional services,Reversal,Salary and wages,Savings and Investments,Shopping,"Transport, Travel, and Logistics"
0,2014-02-28 00:00:00X3000000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2014-02-28 00:00:00X7500000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2014-03-08 00:00:00X4500002863,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2014-03-08 00:00:00X6000002863,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2014-03-08 00:00:00X6000005726,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


((57974, 8), (22625, 8), (22625, 21))

In [3]:
# Wnull values
print('Missing values for Train:', Train.isna().sum())
print('')
print('Missing values for Test:', Test.isna().sum())

Missing values for Train: DATE                       0
TRANSACTION DETAILS     1651
Account_NO                 0
CHQ.NO.                57586
WITHDRAWAL AMT             0
DEPOSIT AMT                0
BALANCE AMT                0
Category                1771
dtype: int64

Missing values for Test: DATE                       0
TRANSACTION DETAILS        0
Account_NO                 0
CHQ.NO.                22504
WITHDRAWAL AMT             0
DEPOSIT AMT                0
BALANCE AMT                0
ID                         0
dtype: int64


In [4]:
# Train.drop(columns='CHQ.NO.', inplace=True)
# Test.drop(columns='CHQ.NO.', inplace=True)


Train['CHQ.NO.'].fillna(-1, inplace=True)
Test['CHQ.NO.'].fillna(-1, inplace=True)
Train['TRANSACTION DETAILS'].fillna("", inplace=True)
Train = Train[Train.Category.notnull()].reset_index(drop=True)

Train.shape, Test.shape

((56203, 8), (22625, 8))

In [5]:
ID = Test.ID
all_data = pd.concat([
        Train.assign(train=1),
        Test.drop(columns='ID').assign(train=0)
])

display(all_data.head(2), all_data.tail(2), all_data.shape)

,DATE,TRANSACTION DETAILS,Account_NO,CHQ.NO.,WITHDRAWAL AMT,DEPOSIT AMT,BALANCE AMT,Category,train
0,2014-02-28,NEFT/FDRL401249529/INDIAFORENSIC,84903292,-1.0,0.0,15000000.0,15000000.0,Money-Transfer,1
1,2014-02-28,NEFT/FDRL401249532/INDIAFORENSIC,84903292,-1.0,0.0,15000000.0,60000000.0,Money-Transfer,1


,DATE,TRANSACTION DETAILS,Account_NO,CHQ.NO.,WITHDRAWAL AMT,DEPOSIT AMT,BALANCE AMT,Category,train
22623,2021-04-16,INDIAFORENSICRUPSETT050319,4969373,-1.0,6822.69,0.00,24431291.37,NaN,0
22624,2021-04-16,INDIAFORENSICRUPSETT050319,4969373,-1.0,0.00,84.38,64122.17,NaN,0


(78828, 9)

In [6]:
## date features

all_data['DATE'] = pd.to_datetime(all_data.DATE)

all_data['month'] = all_data.DATE.dt.month
all_data['day'] = all_data.DATE.dt.day
all_data['year'] = all_data.DATE.dt.year
all_data['dayofweek'] = all_data.DATE.dt.dayofweek
all_data['dayofyear'] = all_data.DATE.dt.dayofyear
all_data['quarter'] = all_data.DATE.dt.quarter
all_data['weekofyear'] = all_data.DATE.dt.weekofyear

all_data.drop(columns='DATE', inplace=True)

all_data.head()

,TRANSACTION DETAILS,Account_NO,CHQ.NO.,WITHDRAWAL AMT,DEPOSIT AMT,BALANCE AMT,Category,train,month,day,year,dayofweek,dayofyear,quarter,weekofyear
0,NEFT/FDRL401249529/INDIAFORENSIC,84903292,-1.0,0.0,15000000.0,15000000.0,Money-Transfer,1,2,28,2014,4,59,1,9
1,NEFT/FDRL401249532/INDIAFORENSIC,84903292,-1.0,0.0,15000000.0,60000000.0,Money-Transfer,1,2,28,2014,4,59,1,9
2,NEFT/FDRL401249534/INDIAFORENSIC,84903292,-1.0,0.0,15000000.0,90000000.0,Money-Transfer,1,2,28,2014,4,59,1,9
3,NEFT/FDRL401249590/INDIAFORENSIC,84903292,-1.0,0.0,10000000.0,100000000.0,Money-Transfer,1,2,28,2014,4,59,1,9
4,NEFT/FDRL401249531/INDIAFORENSIC,84903292,-1.0,0.0,15000000.0,45000000.0,Money-Transfer,1,2,28,2014,4,59,1,9


In [7]:
# type of the transaction
all_data['is_deposit'] = (all_data['DEPOSIT AMT']>0).astype(int)
all_data['is_withdraw'] = (all_data['DEPOSIT AMT']>0).astype(int)
# all_data['WITHDRAWAL AMT'].loc[all_data['WITHDRAWAL AMT'] > 150000000]  = 150000000
# all_data['DEPOSIT AMT'].loc[all_data['DEPOSIT AMT'] > 120000000]  = 120000000
all_data['deposit_balance'] = all_data['DEPOSIT AMT'] / all_data['BALANCE AMT']
all_data['withdraw_balance'] = all_data['WITHDRAWAL AMT'] / all_data['BALANCE AMT']
# all_data['bef_deposit'] = all_data['BALANCE AMT'] - all_data['DEPOSIT AMT']
# all_data['bef_withdrawal'] = all_data['BALANCE AMT'] + all_data['WITHDRAWAL AMT']

all_data['gb_feature_account_blance'] = all_data.groupby(['Account_NO'])['BALANCE AMT'].transform('mean')
# all_data['gb_feature_account_balnce_diff'] = all_data['BALANCE AMT'] - all_data['gb_feature_account_blance']

all_data['gb_feature_account_WITHDRAWAL'] = all_data.groupby(['Account_NO'])['WITHDRAWAL AMT'].transform('mean')
# all_data['gb_feature_account_WITHDRAWAL_diff'] = all_data['WITHDRAWAL AMT'] - all_data['gb_feature_account_WITHDRAWAL']

all_data['gb_feature_account_DEPOSIT'] = all_data.groupby(['Account_NO'])['DEPOSIT AMT'].transform('mean')
# all_data['gb_feature_account_DEPOSI_diff'] = all_data['DEPOSIT AMT'] - all_data['gb_feature_account_DEPOSIT']





all_data.tail(2)

,TRANSACTION DETAILS,Account_NO,CHQ.NO.,WITHDRAWAL AMT,DEPOSIT AMT,BALANCE AMT,Category,train,month,day,year,dayofweek,dayofyear,quarter,weekofyear,is_deposit,is_withdraw,deposit_balance,withdraw_balance,gb_feature_account_blance,gb_feature_account_WITHDRAWAL,gb_feature_account_DEPOSIT
22623,INDIAFORENSICRUPSETT050319,4969373,-1.0,6822.69,0.00,24431291.37,NaN,0,4,16,2021,4,106,2,15,0,0,0.000000,0.000279,1.067709e+07,4563.644801,11.854572
22624,INDIAFORENSICRUPSETT050319,4969373,-1.0,0.00,84.38,64122.17,NaN,0,4,16,2021,4,106,2,15,1,1,0.001316,0.000000,1.067709e+07,4563.644801,11.854572


In [8]:
all_data.Account_NO.value_counts()

20179201    32626
56986993    14643
84903292    12196
80157410     8399
4969373      5374
28039243     3324
10908140      774
68304903      767
83234289      701
22937010       24
Name: Account_NO, dtype: int64

In [9]:
# since this is a categorical let's encode it

all_data['Account_NO'] = le.fit_transform(all_data.Account_NO)

In [10]:
all_data['TRANSACTION DETAILS'].value_counts()[:20]

INTERNALFUNDTRANSFERIN          3262
CASHDEP/GURGAON/                1837
TRFFROMINDIAFORENSICSERVICES    1746
                                1651
CASHDEP/KAROLBAGH/              1649
CASHDEP/RAJOURI/                1538
CASHDEP/NOIDA/TP                1454
INDIAFORENSICAEPSNPCIWDLSET      974
SWEEPTRFTO:40900036427           885
CASHDEP/NEW-DELHI/               723
CASHDEP/FARIDABAD/TP             702
INDIAFORENSICAEPSNPCIDEPSET      638
CASHDEP/SILVASSA/                625
TRFTOINDIAFORENSICSERVICESIN     598
CASHDEP/VIKASMARG/TP             593
CASHDEP/BHI-RIICO/               545
CASHDEP/PANIPAT/0146             534
CASHDEP/GURGAON/LCT              516
CASHDEP/PITAMPURA/TP             488
CASHDEP/KALYAN-W/                441
Name: TRANSACTION DETAILS, dtype: int64

In [11]:
all_data['TRANSACTION DETAILS WORD'] = all_data['TRANSACTION DETAILS'].str\
    .replace('/', " ")
all_data.sample(5)

,TRANSACTION DETAILS,Account_NO,CHQ.NO.,WITHDRAWAL AMT,DEPOSIT AMT,BALANCE AMT,Category,train,month,day,year,dayofweek,dayofyear,quarter,weekofyear,is_deposit,is_withdraw,deposit_balance,withdraw_balance,gb_feature_account_blance,gb_feature_account_WITHDRAWAL,gb_feature_account_DEPOSIT,TRANSACTION DETAILS WORD
32364,CASHDEP/BHI-RIICO/,2,-1.0,0.00,209500.0,3.898351e+10,Deposit,1,12,9,2016,4,344,4,49,1,1,5.374068e-06,0.000000,2.475326e+10,781090.752503,1.754307e+06,CASHDEP BHI-RIICO
4988,CASHDEP/GURGAON/,2,-1.0,0.00,950000.0,4.541416e+09,NaN,0,10,7,2015,2,280,4,41,1,1,2.091859e-04,0.000000,2.475326e+10,781090.752503,1.754307e+06,CASHDEP GURGAON
53624,INDIAFORENSICRUPAYSTLINTC1109,0,-1.0,25.87,0.0,7.061784e+06,Merchant-Payment,1,10,23,2020,4,297,4,43,0,0,0.000000e+00,0.000004,1.067709e+07,4563.644801,1.185457e+01,INDIAFORENSICRUPAYSTLINTC1109
16948,CR800216339053/RMCPL18576/0,9,-1.0,7000.00,0.0,6.038465e+09,Money-Transfer,1,1,9,2016,5,9,1,1,0,0,0.000000e+00,0.000001,7.242483e+09,736223.464530,1.328952e+06,CR800216339053 RMCPL18576 0
41961,NEFT/BARBT17352668050/MAK,2,-1.0,0.00,50025.0,5.158034e+10,Money-Transfer,1,9,13,2017,2,256,3,37,1,1,9.698462e-07,0.000000,2.475326e+10,781090.752503,1.754307e+06,NEFT BARBT17352668050 MAK


In [12]:
wb_vector = TfidfVectorizer(analyzer='char_wb',ngram_range=(1,9))
X_vect = wb_vector.fit_transform(all_data['TRANSACTION DETAILS'])

X_vect.shape

(78828, 1254447)

In [13]:
n_fts = 5
trunc = TruncatedSVD(n_fts)

trunc_fts = trunc.fit_transform(X_vect)
for i in range(n_fts):
    all_data[f"transction_detail_truncft_{i+1}"] = trunc_fts[:, i]

all_data.sample()

,TRANSACTION DETAILS,Account_NO,CHQ.NO.,WITHDRAWAL AMT,DEPOSIT AMT,BALANCE AMT,Category,train,month,day,year,dayofweek,dayofyear,quarter,weekofyear,is_deposit,is_withdraw,deposit_balance,withdraw_balance,gb_feature_account_blance,gb_feature_account_WITHDRAWAL,gb_feature_account_DEPOSIT,TRANSACTION DETAILS WORD,transction_detail_truncft_1,transction_detail_truncft_2,transction_detail_truncft_3,transction_detail_truncft_4,transction_detail_truncft_5
13405,NEFT/FDRL504548553/INDIAFORENSIC,5,-1.0,0.0,10000000.0,4.884147e+10,NaN,0,11,21,2016,0,326,4,47,1,1,0.000205,0.0,4.709821e+10,6.558444e+06,3.560254e+06,NEFT FDRL504548553 INDIAFORENSIC,0.113192,-0.015292,0.171326,0.001746,-0.007025


In [14]:
del trunc, trunc_fts
gc.collect()

0

In [15]:
# n_topics = 5
# LDA = LatentDirichletAllocation(
#     n_components=n_topics, max_iter=10, random_state=42, n_jobs=-1,
#     learning_method="online", batch_size=64
#     )

# lda_trans = LDA.fit_transform(X_vect)

# lda_trans.shape

In [16]:
# distortions = []
# R = range(8, 11)
# for i in R:
#     km = KMeans(
#         n_clusters=i, init='random',
#         n_init=4, max_iter=1000,
#         tol=1e-04, random_state=0
#     )
#     km.fit(X_vect)
#     distortions.append(km.inertia_)

# # plot
# plt.figure(figsize=(8,3))
# plt.plot(R, distortions, marker='o')
# plt.xlabel('Number of clusters')
# plt.ylabel('Distortion')
# plt.show()

In [17]:
classes = 10
km = KMeans(
        n_clusters=classes, init='random',
        n_init=4, max_iter=1000,
        tol=1e-04, random_state=0
    )
km.fit(X_vect)
ktrans = km.predict(X_vect)
all_data['transction_detail_kmeans'] = ktrans

In [18]:
del km, ktrans, X_vect
gc.collect()

0

In [19]:
wb_vector = TfidfVectorizer(analyzer='word',ngram_range=(1, 3))
X_vect = wb_vector.fit_transform(all_data['TRANSACTION DETAILS WORD'])

X_vect.shape

(78828, 111338)

In [20]:
n_fts = 3
trunc = TruncatedSVD(n_fts)

trunc_fts = trunc.fit_transform(X_vect)
for i in range(n_fts):
    all_data[f"transction_detail_word_truncft_{i+1}"] = trunc_fts[:, i]

all_data.sample()

,TRANSACTION DETAILS,Account_NO,CHQ.NO.,WITHDRAWAL AMT,DEPOSIT AMT,BALANCE AMT,Category,train,month,day,year,dayofweek,dayofyear,quarter,weekofyear,is_deposit,is_withdraw,deposit_balance,withdraw_balance,gb_feature_account_blance,gb_feature_account_WITHDRAWAL,gb_feature_account_DEPOSIT,TRANSACTION DETAILS WORD,transction_detail_truncft_1,transction_detail_truncft_2,transction_detail_truncft_3,transction_detail_truncft_4,transction_detail_truncft_5,transction_detail_kmeans,transction_detail_word_truncft_1,transction_detail_word_truncft_2,transction_detail_word_truncft_3
55311,INDIAFORENSICNFSSETT18-12-201,0,-1.0,100.0,0.0,18562750.06,Money-Transfer,1,1,31,2021,6,31,1,4,0,0,0.0,0.000005,1.067709e+07,4563.644801,11.854572,INDIAFORENSICNFSSETT18-12-201,0.189016,-0.019097,0.338941,0.009058,-0.027751,6,0.000002,-0.000012,0.000064


In [21]:
del trunc, trunc_fts, X_vect

gc.collect()

0

In [37]:
lcols = all_data.select_dtypes(['object']).columns.drop("Category").to_list()

lcols

['TRANSACTION DETAILS', 'TRANSACTION DETAILS WORD']

In [38]:
for col in lcols:
    all_data[col] = le.fit_transform(all_data[col])

In [41]:
trans_groups = all_data.groupby('TRANSACTION DETAILS').agg(


             WITHDRAWAL_AMT_min = ("WITHDRAWAL AMT", "min"),
             WITHDRAWAL_AMT_max = ("WITHDRAWAL AMT", "max"),        
#              WITHDRAWAL_AMT_sum = ("WITHDRAWAL AMT", "sum"),
             WITHDRAWAL_AMT_mean = ("WITHDRAWAL AMT", "mean"),
#              WITHDRAWAL_AMT_skew = ("WITHDRAWAL AMT", "skew"),
#              WITHDRAWAL_AMT_std = ("WITHDRAWAL AMT", "std"),
    
             DEPOSIT_AMT_min = ("DEPOSIT AMT", "min"),
             DEPOSIT_AMT_max = ("DEPOSIT AMT", "max"),        
#              DEPOSIT_AMT_sum = ("DEPOSIT AMT", "sum"),
             DEPOSIT_AMT_mean = ("DEPOSIT AMT", "mean"),
#              DEPOSIT_AMT_skew = ("DEPOSIT AMT", "skew"),
#              DEPOSIT_AMT_std = ("DEPOSIT AMT", "std"),
    
             BALANCE_AMT_min = ("BALANCE AMT", "min"),
             BALANCE_AMT_max = ("BALANCE AMT", "max"),        
#              BALANCE_AMT_sum = ("BALANCE AMT", "sum"),
             BALANCE_AMT_mean = ("BALANCE AMT", "mean"),    
#              BALANCE_AMT_skew = ("BALANCE AMT", "skew"),
#              BALANCE_AMT_std = ("BALANCE AMT", "std"),
    
             ).reset_index()

all_data = pd.merge(all_data, trans_groups,on=['TRANSACTION DETAILS'], how ='left')

In [42]:
all_data.isnull().sum()

TRANSACTION DETAILS                     0
Account_NO                              0
CHQ.NO.                                 0
WITHDRAWAL AMT                          0
DEPOSIT AMT                             0
BALANCE AMT                             0
Category                            22625
train                                   0
month                                   0
day                                     0
year                                    0
dayofweek                               0
dayofyear                               0
quarter                                 0
weekofyear                              0
is_deposit                              0
is_withdraw                             0
deposit_balance                         0
withdraw_balance                        0
gb_feature_account_blance               0
gb_feature_account_WITHDRAWAL           0
gb_feature_account_DEPOSIT              0
TRANSACTION DETAILS WORD                0
transction_detail_truncft_1       

In [43]:
train_df = all_data[all_data.train == 1].drop(columns='train')
test_df = all_data[all_data.train == 0].drop(columns=['train', 'Category'])

test_df.shape, train_df.shape

((22625, 39), (56203, 40))

In [44]:
## modeling

train_df['Category'] = le.fit_transform(train_df['Category'])
cat_map = dict(zip(range(len(le.classes_)) , le.classes_))

cat_map

{0: 'Bank Charges',
 1: 'Bill-Payments',
 2: 'Cash-Pickup',
 3: 'Cell Phone and Airtime',
 4: 'Cheque-Payment',
 5: 'Deposit',
 6: 'Donations',
 7: 'Entertainment',
 8: 'General Purchases',
 9: 'Insurance',
 10: 'Interest',
 11: 'Internet and IT Services',
 12: 'Loan Repayment',
 13: 'Merchant-Payment',
 14: 'Money-Transfer',
 15: 'Professional services',
 16: 'Reversal',
 17: 'Salary and wages',
 18: 'Savings and Investments',
 19: 'Shopping',
 20: 'Transport, Travel, and Logistics'}

In [45]:
main_cols = train_df.columns.difference([
    'Category', 
    # 'TRANSACTION DETAILS', 'TRANSACTION DETAILS WORD'
    
'is_withdraw',

'quarter',
'is_deposit',
'dm-Loan_Repayment',
'dm-Internet_and_IT_Services',
'dm-Cash_Pickup',
    
])

X = train_df[main_cols] 
y = train_df['Category']

test = test_df[main_cols]

X.shape, test.shape

((56203, 36), (22625, 36))

In [46]:
from imblearn import under_sampling as uns 


stra1 = {14: 4000, 5: 3000, 13: 1200, 4: 1000, 0: 900,
        2: 850, 17: 700, 1: 400, 19: 119, 3: 103,
        10: 70, 12: 69, 16: 66, 11: 22, 9: 20, 15: 15,
        18: 10, 8: 4, 20: 2, 6: 2, 7: 1}

stra2 = {14: 4000, 5: 3000, 13: 1200, 4: 1000, 0: 900,
        2: 850, 17: 700, 1: 400, 19: 130, 3: 110,
        10: 100, 12: 80, 16: 80, 11: 50, 9: 45, 15: 40,
        18: 30, 8: 20, 20: 15, 6: 15, 7: 10}
        
undersample = uns.RandomUnderSampler(sampling_strategy=stra1, random_state=0)
X, y = undersample.fit_resample(X, y)

oversample = ovs.RandomOverSampler(sampling_strategy=stra2, random_state=0)
X, y = oversample.fit_resample(X, y)

X.shape

(12775, 36)

In [47]:

# class for training more than one models and averging the results
class My_model:
    def __init__(self, models):
        self.models = models

    def fit_eval_pred(self, X, y, eval_set, val, test, verbose=True):
        results_eval = []
        results_test = []
        for i, model in enumerate(self.models):
            if verbose:
                print(f"[Training]............................... Model_{i+1}")
            st = time.time()
            try:
                model.fit(X, y, eval_set=eval_set, verbose=False)
            except Exception as e:
                model.fit(X, y)
            if verbose:
                print(f"[Prediction]............................. Model_{i+1}")
            p = model.predict_proba(val)
            results_eval.append(p)
            results_test.append(model.predict_proba(test))
            if verbose:
                print(f"Log loss = {log_loss(eval_set[0][1], p)}", end=" "*6)
                print(f"Time {time.time() - st :.2f}sec")

        results_eval = np.mean(results_eval, axis=0)
        results_test = np.mean(results_test, axis=0)

        return results_eval, results_test


In [48]:
from sklearn.ensemble import *
from sklearn.multiclass import OneVsRestClassifier
from sklearn.neural_network import MLPClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

# custom function gives models 
def get_models(seed=21):
    rnf_params = {'max_depth': 12, 'max_features':'sqrt', 'n_estimators': 900, 'random_state': seed}
    xgb_params = {"eta":0.1, 'n_estimators': 100, 'random_state':seed, 'objective':'mlogloss',
            "eval_metric":'mlogloss', "subsample" : 1.0, "colsample_bytree" : .8}
    cat_params = {"iterations":1000, "verbose":100, "random_state":seed,
            'use_best_model':True, 'early_stopping_rounds':100, 
        #     "learning_rate":0.09,
            }
    lgb_params = {'learning_rate':0.01, 'n_estimators':10000, 'random_state':seed,
            'early_stopping_round':100, 'verbose':-1, 'subsample':.8, 'colsample_bytree':0.6}

    et_params = {'n_estimators':1000, 'max_features': 'sqrt', 'max_depth':16, 'random_state':seed,
                  'min_samples_leaf':2,  'verbose':0, 'n_jobs': -1}
    bag_params = {'n_jobs':-1, 'max_features':1.0, 'random_state':seed}

    _models = [
            # RandomForestClassifier(100, random_state=seed, n_jobs=-1),
            CatBoostClassifier(**cat_params),   
            LGBMClassifier(**lgb_params),
            XGBClassifier(**xgb_params),
        #     RandomForestClassifier(**rnf_params),
        #     OneVsRestClassifier(ExtraTreesClassifier(**et_params)),
        #     BaggingClassifier(GradientBoostingClassifier(max_features='log2'), n_jobs=-1, max_features=1.0, random_state=seed)
    ]
    return _models


In [ ]:
from sklearn.model_selection import StratifiedKFold

results = []
ns = 5
tot = []


skf = StratifiedKFold(n_splits=ns, random_state=seed, shuffle=True)
for fold, (train_index, test_index) in enumerate(skf.split(X, y)):
    print(100*"-")
    print(f"Fold-{fold+1}")
    X_train, X_test = X.loc[train_index], X.loc[test_index]
    y_train, y_test = y.loc[train_index], y.loc[test_index]

    main_model = My_model(get_models())
    pred_train, pred_test = main_model.fit_eval_pred(X_train, y_train, [(X_test, y_test)], X_test, test)
    try:
      loss = log_loss(y_test, pred_train)
      print(f"log loss = {loss}")
      tot += [loss]
    except Exception as e:
      print(e)
    results.append(pred_test)

    print(100*"-")
    print("\n\n")

print(f"Average log loss = {sum(tot) / len(tot)}")

----------------------------------------------------------------------------------------------------
Fold-1
[Training]............................... Model_1
[Prediction]............................. Model_1
Log loss = 0.024230721684635862      Time 143.78sec
[Training]............................... Model_2


0.009403055530594626

In [ ]:
preds = np.mean(results, axis=0)
preds.shape

In [ ]:
sub = pd.DataFrame(np.round(preds, 4), columns=cat_map.values(), index=ID)
sub.drop(columns="Entertainment", inplace=True)
sub.to_csv("new_base4.csv")
sub.head()